In [2]:
import tensorflow as tf
from tensorflow.keras import Input  # Pastikan Input diimpor
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

2024-11-26 20:15:30.580290: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-26 20:15:30.716947: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732626930.778008    5509 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732626930.794538    5509 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-26 20:15:30.927123: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

In [1]:
import zipfile

# Path ke file zip yang ingin diekstrak
zip_path = 'data.zip'

# Path direktori tujuan untuk hasil ekstraksi
extract_to_path = 'extracted_data/'

# Ekstrak file zip
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

print("File berhasil diekstrak!")

File berhasil diekstrak!


In [4]:
# Path ke folder dataset
train_dir = 'extracted_data/Data Penyakit/train'
validation_dir = 'extracted_data/Data Penyakit/valid'

In [5]:
validation_datagen = ImageDataGenerator(rescale=1.0/255)

In [6]:
# Preprocessing data menggunakan ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [7]:
# Membuat generator untuk data training dan validasi
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),  # Sesuaikan dengan ukuran input yang diinginkan
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

Found 70295 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.


In [8]:
# Mendapatkan nama-nama kelas
class_names = list(train_generator.class_indices.keys())
print("Nama-nama kelas:", class_names)


Nama-nama kelas: ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'T

In [9]:
# Memuat model ResNet50 pre-trained tanpa top layer (include_top=False)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))


I0000 00:00:1732627029.635104    5509 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2159 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [10]:
# Membekukan semua layer dari base model
for layer in base_model.layers:
    layer.trainable = False


In [11]:
# Menambahkan lapisan kustom
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Menambahkan global average pooling layer
x = Dense(1024, activation='relu')(x)  # Lapisan dense kustom
x = Dropout(0.5)(x)  # Dropout untuk regularisasi
predictions = Dense(train_generator.num_classes, activation='softmax')(x)  # Jumlah kelas sesuai dataset Anda

In [12]:
# Membuat model akhir
model = Model(inputs=base_model.input, outputs=predictions)

In [13]:
# Menyusun ulang model dengan lapisan yang di-unfreeze
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0020), 
              loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
# Callback EarlyStopping untuk menghentikan training jika akurasi validasi tidak meningkat
early_stopping = EarlyStopping(
    monitor='val_accuracy', 
    patience=3, 
    mode='max',
    verbose=1,
    restore_best_weights=True
)

In [14]:
# Melatih model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=50,
    callbacks=[early_stopping]  # Callback EarlyStopping
)

/home/wan/anaconda3/envs/tf2/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50


I0000 00:00:1732102835.151200    5046 service.cc:148] XLA service 0x7f20b4015120 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732102835.151409    5046 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2024-11-20 18:40:35.350595: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1732102836.213297    5046 cuda_dnn.cc:529] Loaded cuDNN version 90300
2024-11-20 18:40:37.531732: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_5679_0', 112 bytes spill stores, 224 bytes spill loads

2024-11-20 18:40:37.992234: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_

   3/2197 ━━━━━━━━━━━━━━━━━━━━ 2:35 71ms/step - accuracy: 0.0122 - loss: 4.1219     

I0000 00:00:1732102843.084990    5046 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1306/2197 ━━━━━━━━━━━━━━━━━━━━ 1:38 111ms/step - accuracy: 0.0443 - loss: 3.5906

2024-11-20 18:43:10.301160: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_5679', 16 bytes spill stores, 16 bytes spill loads

2024-11-20 18:43:10.367743: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_5679', 88 bytes spill stores, 88 bytes spill loads

2024-11-20 18:43:10.699960: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_6267', 32 bytes spill stores, 32 bytes spill loads

2024-11-20 18:43:11.921867: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_5679_0', 640 bytes spill stores, 1004 bytes spill loads

2024-11-20 18:43:12.120022: I external/local_xla/xla/st

2197/2197 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - accuracy: 0.0489 - loss: 3.5438

/home/wan/anaconda3/envs/tf2/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-11-20 18:44:55.419813: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1717', 100 bytes spill stores, 100 bytes spill loads

2024-11-20 18:44:55.434488: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1717', 4 bytes spill stores, 4 bytes spill loads

2024-11-20 18:44:55.515958: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to

2197/2197 ━━━━━━━━━━━━━━━━━━━━ 294s 128ms/step - accuracy: 0.0489 - loss: 3.5438 - val_accuracy: 0.0987 - val_loss: 3.3048
Epoch 2/50
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 268s 122ms/step - accuracy: 0.0893 - loss: 3.3188 - val_accuracy: 0.1217 - val_loss: 3.2365
Epoch 3/50
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 255s 116ms/step - accuracy: 0.1074 - loss: 3.2252 - val_accuracy: 0.1465 - val_loss: 3.0338
Epoch 4/50
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 261s 119ms/step - accuracy: 0.1242 - loss: 3.1598 - val_accuracy: 0.1920 - val_loss: 2.8921
Epoch 5/50
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 275s 125ms/step - accuracy: 0.1354 - loss: 3.1004 - val_accuracy: 0.1821 - val_loss: 2.9207
Epoch 6/50
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 274s 125ms/step - accuracy: 0.1432 - loss: 3.0687 - val_accuracy: 0.1689 - val_loss: 3.0041
Epoch 7/50
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 281s 128ms/step - accuracy: 0.1462 - loss: 3.0500 - val_accuracy: 0.2013 - val_loss: 2.8503
Epoch 8/50
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 285s 129ms/step - accuracy: 0.1

In [ ]:
# Meng-unfreeze beberapa lapisan terakhir dari ResNet50 untuk fine-tuning
for layer in base_model.layers[-10:]:  # Meng-unfreeze 10 lapisan terakhir
    layer.trainable = True

In [ ]:
# Menyusun ulang model dengan lapisan yang di-unfreeze
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002), 
              loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Melakukan fine-tuning
history_fine_tuning = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=50,
    callbacks=[early_stopping]  # Gunakan EarlyStopping lagi jika perlu
)


In [ ]:
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")


In [ ]:
# Membangun model CNN
model = Sequential([
    Input(shape=(128, 128, 3)),  # Tentukan bentuk input di sini
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')  # Jumlah kelas sesuai dataset
])

In [ ]:
# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Modifikasi pelatihan model dengan EarlyStopping callback
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=50,
    callbacks=[early_stopping]
)

In [ ]:
import matplotlib.pyplot as plt

# Plot akurasi
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.grid()
plt.show()


In [ ]:
# Simpan model
model.save('/content/image_classification_model.h5')

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf

# Path ke direktori test
test_dir = '/content/test/test'

# Load model
model = tf.keras.models.load_model('image_classification_model.h5')

# Mendapatkan label kelas dari model
class_indices = train_generator.class_indices  # Mengambil mapping dari train_generator
class_labels = {v: k for k, v in class_indices.items()}  # Membalik mapping menjadi indeks ke nama kelas

# Menyiapkan gambar untuk prediksi
def preprocess_image(image_path, target_size=(128, 128)):
    img = load_img(image_path, target_size=target_size)  # Load gambar
    img_array = img_to_array(img)  # Konversi ke array
    img_array = np.expand_dims(img_array, axis=0)  # Tambahkan dimensi batch
    img_array = img_array / 255.0  # Rescale
    return img_array

# Melakukan prediksi untuk setiap gambar
for file_name in os.listdir(test_dir):
    file_path = os.path.join(test_dir, file_name)
    if os.path.isfile(file_path):
        # Preprocessing gambar
        img_array = preprocess_image(file_path)

        # Prediksi
        prediction = model.predict(img_array)
        predicted_class = np.argmax(prediction, axis=1)[0]
        predicted_label = class_labels[predicted_class]

        # Tampilkan hasil
        print(f"Gambar: {file_name}")
        print(f"  Prediksi: {predicted_label}")
        print(f"  Probabilitas: {prediction[0][predicted_class]:.2f}")
        print("----")
